#### Splitting dataset

In [2]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from rdkit.Chem import rdMolDescriptors as rdescriptors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys    
import pubchempy as pcp
import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
from sklearn.utils import shuffle
import sys
sys.path.insert(0,"./../fip3")
from fip.profiles import CooccurrenceProfile
from fip import profiles
import re
from itertools import chain
from ast import literal_eval

In [3]:
#Loading zinc database from csv 
suppl_csv = pd.read_csv('../data/CP1/ZINC/ZINCrandom.csv', delimiter = ' ')

In [4]:
df_split = np.array_split(suppl_csv, 10)

In [9]:
suppl_csv.head(10)

,Zinc_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,8432508,Cc1ccc(-n2c(=O)c3c(n4c(SCCc5ccccc5)nnc24)-c2cc...,532.229683,6.94842,DPUIXUVMXJXCJR-UHFFFAOYSA-N,DPUIXUVMXJXCJR,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[2, 58, 74, 80, 105, 116, 153, 233, 235, 289, ..."
1,2325339,Cc1ccc(-n2c(COc3ccccc3)nnc2SCC(=O)Nc2ccc3c(c2)...,488.151826,4.65672,HVIYVAXZIXJRQC-UHFFFAOYSA-N,HVIYVAXZIXJRQC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 58, 80, 102, 116, 127, 184, 191, 233, 235..."
2,408636482,COCCNS(=O)(=O)c1ccc(OCC(=O)N2CCC(C(N)=O)CC2)c(...,433.107434,0.36740,RAILYSNQBZZKMQ-UHFFFAOYSA-N,RAILYSNQBZZKMQ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 80, 94, 111, 140, 198, 222, 279, 317, 319..."
3,97995285,Cc1ccc(S(=O)(=O)n2nc(NCc3cccc(Br)c3)nc2NCc2ccc...,588.978270,5.57272,FFUQFTXMHIMSHO-UHFFFAOYSA-N,FFUQFTXMHIMSHO,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 82, 92, 157, 235, 255, 319, 350, 374, 378..."
4,408561088,CCn1c(-c2ccccc2)c(C)s/c1=N\C(=O)c1ccccc1Br,400.024496,5.04852,GMWZBJMZXGQSQF-VZCXRCSSSA-N,GMWZBJMZXGQSQF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[30, 34, 57, 80, 162, 289, 294, 308, 313, 351,..."
5,245238933,Cc1ccc(/C=C2\CCC[C@@H]3C2=NN(C(=O)CN2N=N[C@@H]...,622.269239,6.57304,CQTCUVUJTDZYMC-KACGFLGXSA-N,CQTCUVUJTDZYMC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[5, 17, 25, 80, 84, 154, 172, 221, 235, 281, 3..."
6,44838964,COc1ccc(S(=O)(=O)Nc2cc(S(=O)(=O)NC(C)C)ccc2OC)...,448.052956,2.84470,WGFIGLRQYOKKRV-UHFFFAOYSA-N,WGFIGLRQYOKKRV,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 4, 94, 198, 249, 279, 283, 319, 350, 366, ..."
7,754429679,CC[C@@H](C)c1nc([C@H](C)NC(=O)COc2cccc(C#N)c2)...,327.169525,2.44608,NAHNBUHUUPQFPP-NEPJUHHUSA-N,NAHNBUHUUPQFPP,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 41, 66, 71, 74, 80, 102, 117, 192, 283, 29..."
8,408565832,CCCOc1ccc(/C=C/C(=O)Nc2cccc(Br)c2)cc1OCC,403.078306,5.28850,AYIPCSPMXPSFSA-PKNBQFBNSA-N,AYIPCSPMXPSFSA,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 25, 31, 76, 80, 94, 184, 191, 237, 294, 3..."
9,2879473,COc1cccc([C@H]2C[C@@H](C(F)(F)F)n3nc(C(=O)Nc4c...,542.205337,5.80504,ZNLOJLYSEUNRHG-GGAORHGYSA-N,ZNLOJLYSEUNRHG,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 90, 92, 114, 158, 244, 301, 310, 322, 378..."


### Coocurence profile #0

In [5]:
df_split[0] = df_split[0] [['Bitset']]

In [6]:
df_split[0]

,Bitset
0,"[2, 58, 74, 80, 105, 116, 153, 233, 235, 289, ..."
1,"[41, 58, 80, 102, 116, 127, 184, 191, 233, 235..."
2,"[13, 80, 94, 111, 140, 198, 222, 279, 317, 319..."
3,"[80, 82, 92, 157, 235, 255, 319, 350, 374, 378..."
4,"[30, 34, 57, 80, 162, 289, 294, 308, 313, 351,..."
...,...
37897,"[24, 80, 165, 216, 231, 319, 350, 365, 366, 38..."
37898,"[80, 93, 197, 221, 319, 322, 350, 374, 446, 52..."
37899,"[74, 80, 95, 233, 235, 273, 281, 310, 314, 475..."
37900,"[114, 119, 145, 270, 378, 392, 448, 449, 524, ..."


In [10]:
index = df_split[0].index
index

RangeIndex(start=0, stop=37902, step=1)

In [11]:
for x in index:
    a = df_split[0]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[0]['Bitset'][x] = a

In [12]:
df_split[0]['Bitset'][0]

"'2', '58', '74', '80', '105', '116', '153', '233', '235', '289', '314', '341', '352', '378', '389', '504', '507', '548', '607', '650', '747', '765', '926', '935', '940', '984', '1028', '1039', '1045', '1057', '1060', '1066', '1088', '1110', '1119', '1159', '1162', '1164', '1182', '1197', '1199', '1236', '1243', '1294', '1380', '1407', '1449', '1480', '1514', '1638', '1708', '1722', '1750', '1754', '1823', '1825', '1873', '1875', '1920', '1935', '1993', '2013'"

In [13]:
feature_list0 = []
for x in df_split[0]['Bitset']:
    tup = literal_eval(x)
    feature_list0.append(tup)
feature_list0[-1]

('1',
 '14',
 '80',
 '117',
 '241',
 '291',
 '294',
 '350',
 '383',
 '446',
 '503',
 '514',
 '539',
 '562',
 '585',
 '650',
 '662',
 '675',
 '689',
 '739',
 '800',
 '807',
 '875',
 '926',
 '935',
 '974',
 '1019',
 '1028',
 '1057',
 '1088',
 '1121',
 '1125',
 '1152',
 '1273',
 '1315',
 '1325',
 '1363',
 '1365',
 '1380',
 '1391',
 '1459',
 '1476',
 '1480',
 '1521',
 '1544',
 '1567',
 '1652',
 '1691',
 '1722',
 '1749',
 '1750',
 '1790',
 '1853',
 '1865',
 '1873',
 '1917',
 '2025')

In [14]:
#Subset #0
p0 = CooccurrenceProfile.from_feature_lists(feature_list0)

In [15]:
p0.df

value
feature1 feature2       
1028     1028       7952
         1039       1360
         1045        206
         105         220
         1057       6049
...                  ...
1341     801           1
1939     801           1
270      959           1
449      959           1
1125     503           1

[1671610 rows x 1 columns]

In [17]:
p0.df.to_csv('../data/CP1/ZINC/ZINCp0_37902.csv')

In [18]:
p0 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp0_37902.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37902)
p0.df

value
feature1 feature2       
1028     1028       7952
         1039       1360
         1045        206
         105         220
         1057       6049
...                  ...
1341     801           1
1939     801           1
270      959           1
449      959           1
1125     503           1

[1671610 rows x 1 columns]

In [78]:
p.df.index.dtype

dtype('O')

In [60]:
p.df.inde = p.df.index

In [63]:
p.df.index = p.df.index.map(str)

In [46]:
p.df['value']= p.df['value'].map(str)

In [47]:
p.df.dtypes

value    object
dtype: object

In [25]:
p.add_another_cooccurrence_profile(p)

In [26]:
p.df

value
feature1 feature2       
1        1         18576
         1004        538
         1057      17096
         1063        368
         1087        440
...                  ...
1144     710           2
1269     1759          2
35       710           2
348      618           2
1511     347           2

[1675660 rows x 1 columns]

In [17]:
p0.add_another_cooccurrence_profile(p0)

In [18]:
p0.df

value
feature1 feature2       
1019     1019      30380
         1057      24160
         1145       4922
         1152      14174
         1175        308
...                  ...
2026     924           2
46       924           2
758      924           2
1557     863           2
293      863           2

[1675986 rows x 1 columns]

In [79]:
from fip.profiles import CooccurrenceProbabilityProfile
q = CooccurrenceProbabilityProfile.from_cooccurrence_profile(p)
q

In [80]:
q.df

value
feature1 feature2          
1017     1017      0.257613
         1019      0.199401
         1035      0.098742
         1057      0.255675
         1060      0.157571
...                     ...
1100     748       0.000003
1054     512       0.000003
1246     505       0.000003
135      1757      0.000003
1705     1789      0.000006

[2096060 rows x 1 columns]

In [81]:
from fip.profiles import PointwiseMutualInformationProfile
r = PointwiseMutualInformationProfile.from_cooccurrence_probability_profile(q)
r

In [82]:
r.df

value
feature1 feature2          
1017     1017      0.000000
         1019      0.042145
         1035      1.560974
         1057      0.015667
         1060      0.393332
...                     ...
1100     748      -3.700894
1054     512      -3.712538
1246     505      -4.508552
135      1757     -3.486017
1705     1789     -3.585085

[2096060 rows x 1 columns]

In [652]:
pd.read_csv('../data/CooccurenceProfile/ZINCp0.csv',index_col=[0,1])

value
feature1 feature2       
1011     1011        541
         1039        175
         1057        396
         1088        341
         1238         50
...                  ...
1058     169           1
1473     871           1
1548     169           1
         871           1
2028     898           1

[1674218 rows x 1 columns]

### Coocurence profile #1

In [24]:
df_split[1] = df_split[1] [['Bitset']]
df_split[1]

,Bitset
37902,"'1', '42', '43', '50', '54', '80', '114', '121..."
37903,"'80', '94', '116', '140', '231', '249', '252',..."
37904,"'65', '162', '201', '289', '366', '378', '385'..."
37905,"'1', '41', '61', '75', '80', '90', '154', '191..."
37906,"'1', '13', '41', '69', '80', '83', '94', '114'..."
...,...
75799,"'25', '233', '235', '250', '261', '288', '314'..."
75800,"'19', '41', '80', '116', '191', '246', '310', ..."
75801,"'29', '80', '197', '218', '295', '369', '378',..."
75802,"'5', '80', '87', '116', '148', '310', '314', '..."


In [20]:
index1 = df_split[1].index
index1

RangeIndex(start=37902, stop=75804, step=1)

In [21]:
for x in index1:
    a = df_split[1]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[1]['Bitset'][x] = a

In [ ]:
feature_list1 = []
for x in df_split[1]['Bitset']:
    tup = literal_eval(x)
    feature_list1.append(tup)
feature_list1[:-1]

In [26]:
feature_list1[-1]

('216',
 '246',
 '378',
 '389',
 '401',
 '456',
 '459',
 '496',
 '650',
 '695',
 '699',
 '807',
 '841',
 '855',
 '910',
 '1057',
 '1203',
 '1380',
 '1536',
 '1744',
 '1750',
 '1840',
 '1842',
 '1855',
 '1873',
 '1917',
 '1928')

In [28]:
p1 = CooccurrenceProfile.from_feature_lists(feature_list1)
p1.df

value
feature1 feature2       
1        1          9204
         1028       2111
         1057       8470
         1088       5222
         114         980
...                  ...
1744     459           1
         910           1
459      496           1
         910           1
496      910           1

[1672349 rows x 1 columns]

In [29]:
p1.df.to_csv('../data/CP1/ZINC/ZINCp1_37902.csv')

In [30]:
p1 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp1_37902.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37902)
p1.df

value
feature1 feature2       
1        1          9204
         1028       2111
         1057       8470
         1088       5222
         114         980
...                  ...
1744     459           1
         910           1
459      496           1
         910           1
496      910           1

[1672349 rows x 1 columns]

### Coocurence profile 2

In [31]:
df_split[2] = df_split[2] [['Bitset']]
df_split[2]

,Bitset
75804,"[1, 34, 80, 102, 118, 227, 263, 305, 322, 355,..."
75805,"[5, 133, 216, 233, 314, 561, 650, 656, 787, 81..."
75806,"[74, 80, 101, 145, 147, 216, 294, 301, 314, 32..."
75807,"[4, 165, 193, 216, 319, 322, 350, 366, 389, 50..."
75808,"[1, 26, 74, 283, 352, 378, 615, 715, 739, 786,..."
...,...
113700,"[2, 74, 80, 160, 226, 248, 273, 277, 314, 369,..."
113701,"[4, 113, 180, 305, 314, 319, 350, 497, 562, 60..."
113702,"[1, 34, 35, 74, 80, 90, 199, 231, 283, 294, 31..."
113703,"[25, 80, 94, 125, 145, 147, 162, 237, 246, 281..."


In [32]:
index2 = df_split[2].index
index2

RangeIndex(start=75804, stop=113705, step=1)

In [33]:
for x in index2:
    a = df_split[2]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[2]['Bitset'][x] = a

In [34]:
feature_list2 = []
for x in df_split[2]['Bitset']:
    tup = literal_eval(x)
    feature_list2.append(tup)
feature_list2[-1]

('80',
 '81',
 '113',
 '197',
 '209',
 '215',
 '221',
 '314',
 '352',
 '436',
 '528',
 '585',
 '586',
 '650',
 '667',
 '679',
 '689',
 '734',
 '768',
 '807',
 '855',
 '884',
 '913',
 '926',
 '935',
 '984',
 '1001',
 '1019',
 '1028',
 '1057',
 '1060',
 '1063',
 '1084',
 '1088',
 '1094',
 '1114',
 '1145',
 '1152',
 '1181',
 '1199',
 '1226',
 '1262',
 '1292',
 '1309',
 '1325',
 '1380',
 '1444',
 '1470',
 '1554',
 '1569',
 '1644',
 '1657',
 '1750',
 '1835',
 '1873',
 '1917',
 '1980',
 '1982',
 '2047')

In [35]:
p2 = CooccurrenceProfile.from_feature_lists(feature_list2)
p2.df

value
feature1 feature2       
1        1          9264
         102         624
         1136        480
         1160        892
         118         392
...                  ...
436      528           1
         768           1
         81            1
         884           1
734      768           1

[1673545 rows x 1 columns]

In [36]:
p2.df.to_csv('../data/CP1/ZINC/ZINCp2_37901.csv')

In [37]:
p2 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp2_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p2.df

value
feature1 feature2       
1        1          9264
         102         624
         1136        480
         1160        892
         118         392
...                  ...
436      528           1
         768           1
         81            1
         884           1
734      768           1

[1673545 rows x 1 columns]

### Coocurence profile 3

In [38]:
df_split[3] = df_split[3] [['Bitset']]
df_split[3]

,Bitset
113705,"[2, 90, 119, 209, 273, 390, 394, 534, 605, 650..."
113706,"[74, 105, 168, 304, 322, 378, 383, 552, 695, 7..."
113707,"[9, 80, 102, 151, 166, 272, 387, 389, 430, 433..."
113708,"[14, 97, 110, 171, 177, 184, 191, 245, 314, 31..."
113709,"[33, 80, 114, 162, 245, 251, 291, 319, 350, 37..."
...,...
151601,"[14, 67, 80, 88, 95, 97, 130, 207, 215, 273, 2..."
151602,"[29, 34, 45, 80, 112, 114, 235, 286, 294, 310,..."
151603,"[80, 159, 162, 248, 361, 378, 461, 473, 490, 5..."
151604,"[1, 66, 80, 81, 117, 132, 145, 158, 238, 283, ..."


In [39]:
index3 = df_split[3].index
index3

RangeIndex(start=113705, stop=151606, step=1)

In [40]:
for x in index3:
    a = df_split[3]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[3]['Bitset'][x] = a

In [41]:
feature_list3 = []
for x in df_split[3]['Bitset']:
    tup = literal_eval(x)
    feature_list3.append(tup)
feature_list3[-1]

('94',
 '116',
 '190',
 '191',
 '245',
 '317',
 '387',
 '394',
 '486',
 '530',
 '573',
 '650',
 '796',
 '807',
 '843',
 '849',
 '890',
 '926',
 '1017',
 '1019',
 '1024',
 '1028',
 '1057',
 '1077',
 '1152',
 '1249',
 '1325',
 '1380',
 '1385',
 '1403',
 '1665',
 '1697',
 '1742',
 '1750',
 '1816',
 '1873',
 '1917',
 '2036')

In [42]:
p3 = CooccurrenceProfile.from_feature_lists(feature_list3)
p3.df

value
feature1 feature2       
1014     1014        790
         1019        295
         1028        149
         1083          6
         1126         12
...                  ...
1705     665           1
1877     569           1
1024     190           1
1742     190           1
2036     796           1

[1676212 rows x 1 columns]

In [43]:
p3.df.to_csv('../data/CP1/ZINC/ZINCp3_37901.csv')

In [44]:
p3 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp3_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p3.df

value
feature1 feature2       
1014     1014        790
         1019        295
         1028        149
         1083          6
         1126         12
...                  ...
1705     665           1
1877     569           1
1024     190           1
1742     190           1
2036     796           1

[1676212 rows x 1 columns]

### Coocurence profile 4

In [45]:
df_split[4] = df_split[4] [['Bitset']]
df_split[4]

,Bitset
151606,"[2, 41, 80, 94, 103, 162, 249, 313, 319, 350, ..."
151607,"[30, 80, 119, 127, 155, 216, 235, 289, 294, 31..."
151608,"[1, 102, 113, 283, 305, 314, 334, 352, 410, 45..."
151609,"[80, 118, 191, 216, 276, 369, 446, 451, 464, 4..."
151610,"[58, 80, 116, 230, 233, 235, 318, 341, 378, 38..."
...,...
189502,"[41, 43, 80, 231, 325, 378, 387, 409, 437, 608..."
189503,"[1, 5, 21, 25, 71, 224, 281, 322, 378, 389, 39..."
189504,"[10, 80, 95, 170, 202, 216, 389, 552, 561, 675..."
189505,"[34, 41, 53, 74, 80, 102, 114, 145, 191, 246, ..."


In [46]:
index4 = df_split[4].index
index4

RangeIndex(start=151606, stop=189507, step=1)

In [47]:
for x in index4:
    a = df_split[4]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[4]['Bitset'][x] = a

In [48]:
feature_list4 = []
for x in df_split[4]['Bitset']:
    tup = literal_eval(x)
    feature_list4.append(tup)
feature_list4[-1]

('80',
 '107',
 '140',
 '224',
 '268',
 '289',
 '314',
 '338',
 '350',
 '363',
 '378',
 '504',
 '580',
 '585',
 '629',
 '650',
 '760',
 '763',
 '807',
 '881',
 '921',
 '926',
 '932',
 '935',
 '1000',
 '1009',
 '1057',
 '1088',
 '1097',
 '1163',
 '1199',
 '1312',
 '1313',
 '1357',
 '1375',
 '1380',
 '1435',
 '1452',
 '1455',
 '1476',
 '1480',
 '1573',
 '1589',
 '1750',
 '1873',
 '1917',
 '2009')

In [50]:
p4 = CooccurrenceProfile.from_feature_lists(feature_list4)
p4.df

value
feature1 feature2       
1028     1028       8046
         103          27
         1057       6069
         1058         80
         1127         23
...                  ...
1903     548           1
107      921           1
1435     338           1
         629           1
629      921           1

[1678605 rows x 1 columns]

In [55]:
p4.df.to_csv('../data/CP1/ZINC/ZINCp4_37901.csv')

In [56]:
p4 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp4_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p4.df

value
feature1 feature2       
1028     1028       8046
         103          27
         1057       6069
         1058         80
         1127         23
...                  ...
1903     548           1
107      921           1
1435     338           1
         629           1
629      921           1

[1678605 rows x 1 columns]

### Coocurence profile 5

In [57]:
df_split[5] = df_split[5] [['Bitset']]
df_split[5]

,Bitset
189507,"[13, 76, 80, 191, 237, 294, 322, 333, 425, 428..."
189508,"[80, 196, 242, 350, 378, 389, 456, 464, 585, 6..."
189509,"[1, 34, 42, 50, 74, 80, 83, 140, 162, 216, 268..."
189510,"[43, 80, 123, 217, 330, 378, 407, 435, 459, 52..."
189511,"[27, 41, 60, 80, 102, 114, 143, 197, 305, 355,..."
...,...
227403,"[25, 43, 75, 77, 80, 86, 110, 115, 210, 249, 2..."
227404,"[41, 80, 114, 235, 299, 319, 378, 387, 392, 46..."
227405,"[41, 58, 80, 90, 102, 116, 162, 235, 310, 318,..."
227406,"[1, 165, 216, 222, 389, 483, 484, 561, 592, 87..."


In [58]:
index5 = df_split[5].index
index5

RangeIndex(start=189507, stop=227408, step=1)

In [59]:
for x in index5:
    a = df_split[5]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[5]['Bitset'][x] = a

In [60]:
feature_list5 = []
for x in df_split[5]['Bitset']:
    tup = literal_eval(x)
    feature_list5.append(tup)
feature_list5[-1]

('80',
 '90',
 '92',
 '94',
 '95',
 '131',
 '140',
 '227',
 '233',
 '249',
 '378',
 '389',
 '401',
 '411',
 '650',
 '675',
 '688',
 '695',
 '699',
 '734',
 '736',
 '789',
 '807',
 '841',
 '875',
 '914',
 '927',
 '935',
 '1019',
 '1057',
 '1088',
 '1160',
 '1171',
 '1187',
 '1199',
 '1245',
 '1279',
 '1292',
 '1313',
 '1380',
 '1410',
 '1440',
 '1443',
 '1452',
 '1536',
 '1627',
 '1648',
 '1670',
 '1722',
 '1750',
 '1803',
 '1805',
 '1825',
 '1852',
 '1861',
 '1867',
 '1873',
 '1898',
 '1917',
 '1928',
 '1937',
 '1956',
 '1970',
 '1972',
 '2000',
 '2004')

In [61]:
p5 = CooccurrenceProfile.from_feature_lists(feature_list5)
p5.df

value
feature1 feature2       
1057     1057      30486
         1077       2222
         1152      17744
         1238       4459
         13         2202
...                  ...
1627     734           1
1805     411           1
1861     789           1
1867     688           1
688      914           1

[1677665 rows x 1 columns]

In [62]:
p5.df.to_csv('../data/CP1/ZINC/ZINCp5_37901.csv')

In [63]:
p5 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp5_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p5.df

value
feature1 feature2       
1057     1057      30486
         1077       2222
         1152      17744
         1238       4459
         13         2202
...                  ...
1627     734           1
1805     411           1
1861     789           1
1867     688           1
688      914           1

[1677665 rows x 1 columns]

### Coocurence profile 6

In [64]:
df_split[6] = df_split[6] [['Bitset']]
df_split[6]

,Bitset
227408,"[1, 80, 140, 145, 147, 249, 252, 283, 294, 410..."
227409,"[1, 45, 69, 80, 114, 121, 126, 191, 283, 378, ..."
227410,"[2, 13, 41, 80, 81, 102, 197, 252, 319, 322, 3..."
227411,"[1, 80, 192, 244, 281, 378, 407, 413, 421, 437..."
227412,"[2, 14, 41, 45, 80, 131, 184, 191, 314, 360, 3..."
...,...
265304,"[1, 80, 117, 235, 271, 283, 288, 294, 310, 319..."
265305,"[41, 80, 197, 255, 310, 325, 333, 389, 650, 73..."
265306,"[1, 41, 58, 80, 114, 116, 117, 145, 233, 283, ..."
265307,"[80, 378, 383, 389, 455, 493, 570, 573, 650, 6..."


In [65]:
index6 = df_split[6].index
index6

RangeIndex(start=227408, stop=265309, step=1)

In [66]:
for x in index6:
    a = df_split[6]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[6]['Bitset'][x] = a

In [67]:
feature_list6 = []
for x in df_split[6]['Bitset']:
    tup = literal_eval(x)
    feature_list6.append(tup)
feature_list6[-1]

('29',
 '37',
 '69',
 '80',
 '98',
 '319',
 '333',
 '350',
 '374',
 '378',
 '389',
 '575',
 '585',
 '602',
 '650',
 '674',
 '739',
 '753',
 '757',
 '760',
 '777',
 '800',
 '807',
 '926',
 '935',
 '943',
 '1019',
 '1057',
 '1060',
 '1071',
 '1084',
 '1088',
 '1105',
 '1129',
 '1145',
 '1154',
 '1199',
 '1203',
 '1257',
 '1380',
 '1449',
 '1452',
 '1476',
 '1546',
 '1696',
 '1724',
 '1743',
 '1750',
 '1791',
 '1837',
 '1866',
 '1873',
 '1939',
 '1956',
 '1972',
 '2000')

In [68]:
p6 = CooccurrenceProfile.from_feature_lists(feature_list6)
p6.df

value
feature1 feature2       
1        1          9153
         1039       1554
         1057       8445
         1088       5205
         1144        193
...                  ...
1546     98            1
1743     1837          1
         777           1
         943           1
         98            1

[1673968 rows x 1 columns]

In [69]:
p6.df.to_csv('../data/CP1/ZINC/ZINCp6_37901.csv')

In [70]:
p6 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp6_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p6.df

value
feature1 feature2       
1        1          9153
         1039       1554
         1057       8445
         1088       5205
         1144        193
...                  ...
1546     98            1
1743     1837          1
         777           1
         943           1
         98            1

[1673968 rows x 1 columns]

### Coocurence profile 7

In [71]:
df_split[7] = df_split[7] [['Bitset']]
df_split[7]

,Bitset
265309,"[114, 177, 241, 255, 281, 377, 378, 407, 436, ..."
265310,"[84, 106, 300, 378, 384, 397, 519, 739, 807, 8..."
265311,"[1, 62, 80, 143, 294, 377, 378, 591, 627, 739,..."
265312,"[1, 21, 68, 80, 102, 283, 340, 378, 656, 667, ..."
265313,"[41, 80, 90, 162, 310, 325, 378, 385, 389, 650..."
...,...
303205,"[1, 42, 50, 54, 80, 145, 147, 162, 216, 220, 2..."
303206,"[80, 95, 162, 259, 377, 378, 388, 461, 556, 65..."
303207,"[80, 115, 116, 147, 153, 212, 289, 322, 341, 3..."
303208,"[1, 50, 122, 140, 162, 214, 216, 252, 309, 313..."


In [72]:
index7 = df_split[7].index
index7

RangeIndex(start=265309, stop=303210, step=1)

In [73]:
for x in index7:
    a = df_split[7]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[7]['Bitset'][x] = a

In [74]:
feature_list7 = []
for x in df_split[7]['Bitset']:
    tup = literal_eval(x)
    feature_list7.append(tup)
feature_list7[-1]

('1',
 '41',
 '42',
 '59',
 '80',
 '97',
 '116',
 '117',
 '124',
 '162',
 '283',
 '294',
 '303',
 '341',
 '349',
 '361',
 '378',
 '384',
 '389',
 '511',
 '548',
 '609',
 '650',
 '656',
 '675',
 '739',
 '747',
 '765',
 '794',
 '807',
 '886',
 '889',
 '903',
 '926',
 '935',
 '940',
 '994',
 '997',
 '1004',
 '1017',
 '1045',
 '1057',
 '1060',
 '1066',
 '1088',
 '1152',
 '1164',
 '1182',
 '1199',
 '1208',
 '1243',
 '1274',
 '1380',
 '1393',
 '1407',
 '1430',
 '1449',
 '1506',
 '1533',
 '1597',
 '1658',
 '1696',
 '1750',
 '1754',
 '1785',
 '1818',
 '1852',
 '1855',
 '1873',
 '1917',
 '1943',
 '1959',
 '1984',
 '2018',
 '2032')

In [75]:
p7 = CooccurrenceProfile.from_feature_lists(feature_list7)
p7.df

value
feature1 feature2       
1019     1019      14929
         1057      11808
         114        1402
         1164       1151
         1166        122
...                  ...
1533     994           1
1818     994           1
303      994           1
765      97            1
         994           1

[1677484 rows x 1 columns]

In [77]:
p7.df.to_csv('../data/CP1/ZINC/ZINCp7_37901.csv')

In [78]:
p7 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp7_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p7.df

value
feature1 feature2       
1019     1019      14929
         1057      11808
         114        1402
         1164       1151
         1166        122
...                  ...
1533     994           1
1818     994           1
303      994           1
765      97            1
         994           1

[1677484 rows x 1 columns]

#### Coocurence profile 8

In [79]:
df_split[8] = df_split[8] [['Bitset']]
df_split[8]

,Bitset
303210,"[80, 94, 104, 145, 226, 248, 249, 310, 323, 36..."
303211,"[45, 71, 80, 95, 145, 147, 168, 199, 216, 371,..."
303212,"[1, 80, 145, 155, 184, 191, 283, 294, 310, 372..."
303213,"[29, 80, 90, 140, 174, 233, 407, 484, 593, 624..."
303214,"[1, 41, 58, 74, 80, 86, 97, 116, 117, 222, 283..."
...,...
341106,"[14, 32, 41, 42, 50, 54, 80, 116, 145, 147, 16..."
341107,"[5, 80, 132, 134, 165, 216, 231, 238, 242, 244..."
341108,"[80, 161, 202, 227, 249, 310, 314, 322, 343, 3..."
341109,"[1, 30, 80, 107, 117, 283, 319, 322, 350, 389,..."


In [80]:
index8 = df_split[8].index
index8

RangeIndex(start=303210, stop=341111, step=1)

In [81]:
for x in index8:
    a = df_split[8]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[8]['Bitset'][x] = a

In [82]:
feature_list8 = []
for x in df_split[8]['Bitset']:
    tup = literal_eval(x)
    feature_list8.append(tup)
feature_list8[-1]

('1',
 '13',
 '14',
 '33',
 '45',
 '80',
 '90',
 '158',
 '197',
 '241',
 '256',
 '287',
 '289',
 '310',
 '330',
 '378',
 '383',
 '429',
 '459',
 '486',
 '507',
 '543',
 '573',
 '650',
 '667',
 '675',
 '695',
 '699',
 '760',
 '795',
 '807',
 '841',
 '881',
 '935',
 '1005',
 '1057',
 '1088',
 '1097',
 '1147',
 '1152',
 '1153',
 '1164',
 '1199',
 '1233',
 '1238',
 '1285',
 '1312',
 '1380',
 '1410',
 '1440',
 '1455',
 '1567',
 '1655',
 '1670',
 '1681',
 '1691',
 '1750',
 '1754',
 '1783',
 '1790',
 '1806',
 '1861',
 '1873',
 '1917',
 '1928',
 '1996')

In [84]:
p8 = CooccurrenceProfile.from_feature_lists(feature_list8)
p8.df

value
feature1 feature2       
1017     1017       2184
         1027         95
         1032         34
         1039        356
         104          14
...                  ...
693      826           1
1081     725           1
107      1188          1
1188     2042          1
         30            1

[1676593 rows x 1 columns]

In [85]:
p8.df.to_csv('../data/CP1/ZINC/ZINCp8_37901.csv')

In [86]:
p8 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp8_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p8.df

value
feature1 feature2       
1017     1017       2184
         1027         95
         1032         34
         1039        356
         104          14
...                  ...
693      826           1
1081     725           1
107      1188          1
1188     2042          1
         30            1

[1676593 rows x 1 columns]

### Coocurence profile 9

In [87]:
df_split[9] = df_split[9] [['Bitset']]
df_split[9]

,Bitset
341111,"[1, 53, 80, 117, 184, 188, 217, 246, 283, 310,..."
341112,"[13, 80, 99, 145, 147, 165, 167, 216, 294, 450..."
341113,"[13, 80, 156, 204, 294, 314, 326, 334, 361, 37..."
341114,"[1, 54, 80, 147, 265, 302, 650, 695, 807, 841,..."
341115,"[1, 11, 79, 80, 94, 102, 117, 188, 191, 305, 3..."
...,...
379007,"[25, 80, 94, 125, 145, 147, 162, 237, 249, 281..."
379008,"[13, 76, 80, 94, 102, 197, 255, 265, 294, 322,..."
379009,"[80, 105, 257, 378, 437, 446, 736, 739, 744, 7..."
379010,"[1, 46, 94, 117, 235, 249, 389, 484, 486, 556,..."


In [88]:
index9 = df_split[9].index
index9

RangeIndex(start=341111, stop=379012, step=1)

In [89]:
for x in index9:
    a = df_split[9]['Bitset'][x]
    a = a.split(", ")
    a = re.sub('\[', '', str(a))
    a = re.sub('\]', '', str(a))
    df_split[9]['Bitset'][x] = a

In [90]:
feature_list9 = []
for x in df_split[9]['Bitset']:
    tup = literal_eval(x)
    feature_list9.append(tup)
feature_list9[-1]

('74',
 '148',
 '207',
 '362',
 '389',
 '469',
 '552',
 '553',
 '650',
 '807',
 '891',
 '906',
 '926',
 '1019',
 '1088',
 '1114',
 '1122',
 '1325',
 '1362',
 '1431',
 '1507',
 '1638',
 '1917')

In [91]:
p9 = CooccurrenceProfile.from_feature_lists(feature_list9)
p9.df

value
feature1 feature2       
1        1          9186
         1019       3088
         1028       2145
         1039       1543
         1057       8521
...                  ...
578      595           1
         733           1
595      914           1
1122     553           1
553      891           1

[1673651 rows x 1 columns]

In [92]:
p9.df.to_csv('../data/CP1/ZINC/ZINCp9_37901.csv')

In [93]:
p9 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp9_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p9.df

value
feature1 feature2       
1        1          9186
         1019       3088
         1028       2145
         1039       1543
         1057       8521
...                  ...
578      595           1
         733           1
595      914           1
1122     553           1
553      891           1

[1673651 rows x 1 columns]

In [ ]:
p0.df.add_another_cooccurrence_profile(zinc1)